In [25]:
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types
import re
import subprocess

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [26]:
import json

# Path to your JSON file
file_path = "Datasets/file_analysis_2/fa2.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [24]:
def ExecuteCmd(command_str) -> str:
    """Execute a shell command and save its output to a file.

    Args:
        command_str: The shell command to execute as a string.

    Returns:
        A message indicating that the output was saved in a file.
    """
    output_file = 'output_cmd/output.txt'
    with open(output_file, 'w') as f:
        subprocess.run(
            command_str,
            shell=True,
            stderr=f,
            stdout=f,
            text=True,
            executable='/bin/bash'
        )
    return f"Output saved in a file"


# Tool 1: get the next question from a list
def next_question() -> str:
    """This tool reads and returns the next question and can be used to get the first question."""
    global i, questions
    i += 1
    return questions[i-1]

# Tool 2: read the command output file
def read_output() -> str:
    """This tool reads the file where command outputs are stored and returns its content(may be truncated if output is very long)."""
    output_file = 'output_cmd/output.txt'
    with open(output_file, 'r') as f:
        output = f.read()
    if len(output) > 500:
        output = output[:500] + "\n...[truncated]..."
    return output.strip()


In [27]:
ExecuteCmd_dec = {
    "name": "ExecuteCmd",
    "description": "Execute a shell command and save its output to a file.",
    "parameters": {
        "type": "object",
        "properties": {
            "command_str": {
                "type": "string",
                "description": "The shell command to execute."
            }
        },
        "required": ["command_str"]
    }
}

next_question_dec = {
    "name": "next_question",
    "description": "Reads and returns the next question and can be used to get the first question.",
    "parameters": {
        "type": "object",
        "properties": {},  # no parameters required
        "required": []
    }
}

read_output_dec = {
    "name": "read_output",
    "description": "Reads the file where command outputs are stored and returns its content. Output may be truncated if very long.",
    "parameters": {
        "type": "object",
        "properties": {},  # no parameters required
        "required": []
    }
}


In [28]:
prompt= f"""
You are tasked with solving a cybersecurity investigation challenge composed of multiple sequential questions. Use only Ubuntu terminal commands to analyze the provided file and answer each question in order.

Scenario:
{scenario}

Available Command-line tools you can use:
{', '.join(tools)}

Target File:
{', '.join(file)} 

You run in a loop of Thought, Action, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the task you have been given.
You must output your detailed reasoning before returning the action.
Use Action to run one of the tools available to you using Function Calling.
Observation will be the result of running those actions.

Your available tools are:
- ExecuteCmd
- next_question
- read_output

Example session:

Task by the user: Take the next question and solve it by providing a linux command

Thought: I need to get the next question
Action: function calling next_question

You will be called again with this:

Observation: the next question

Thought: I need to generate a command to solve the question 
Action: Function Calling ExecuteCmd

You will be called again with this:

Task by the user: Analyze the previous command's output and generate a new command or return the final answer

Thought: i need to see the output of the command
Action: Function Calling read_output

You will be called again with this:

Observation: the output of the command

Thought: I need to check if the answer is in the output, if not i will give another command 
Action: Function Calling ExecuteCmd
OR
Answer: answer to the question

You will be called again with this:
Task by the user: Take the next question and solve it by providing a linux command """.strip()
print (prompt)

You are tasked with solving a cybersecurity investigation challenge composed of multiple sequential questions. Use only Ubuntu terminal commands to analyze the provided file and answer each question in order.

Scenario:
A suspicious Pwershell script in a .txt file was found on one of our endpoints, can you work out what it does?

Available Command-line tools you can use:
grep, awk, sed, sha256sum, base64, cut, sort, wc, head, tail, find, strings, file, stat

Target File:
Datasets/file_analysis_2/HDWallpaperEngine.txt 

You run in a loop of Thought, Action, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the task you have been given.
You must output your detailed reasoning before returning the action.
Use Action to run one of the tools available to you using Function Calling.
Observation will be the result of running those actions.

Your available tools are:
- ExecuteCmd
- next_question
- read_output

Example session:

Task by the us

In [34]:
prompt3= f"""
You are tasked with solving a cybersecurity investigation challenge composed of multiple sequential questions. Use only Ubuntu terminal commands to analyze the provided file and answer each question in order.

Scenario:
{scenario}

Available Command-line tools you can use:
{', '.join(tools)}

Target File:
{', '.join(file)} 

You run in a loop of Thought, Action, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the task you have been given.
You must output your detailed reasoning before returning the action.
Use Action to run one of the tools available to you using Function Calling.
Observation will be the result of running those actions.

Your available tools are:
- ExecuteCmd
- next_question
- read_output """

In [30]:
prompt2="""You are tasked with solving a cybersecurity investigation challenge composed of multiple sequential questions. Your process must follow a strict loop: Thought, Action, Observation. At the end of the loop, you will output an Answer.

**Process Rules:**
1.  **Thought:** You MUST first describe your reasoning and plan. Explain what the goal is and which command you plan to use and why. This is a mandatory step before any action.
2.  **PAUSE** Take a break after generating a thought
3.  **Action:** After the Thought, you will use function calling to execute one of the available tools (dont do this if there isn't a thought in the history).
4.  **Observation:** This will be the result of the action you took.

Use only Ubuntu terminal commands to analyze the provided file and answer each question in order.

**Scenario:**
A suspicious Pwershell script in a .txt file was found on one of our endpoints, can you work out what it does?

**Available Command-line tools:**
grep, awk, sed, sha256sum, base64, cut, sort, wc, head, tail, find, strings, file, stat

**Target File:**
Datasets/file_analysis_2/HDWallpaperEngine.txt

**Available Tools for Function Calling:**
- `ExecuteCmd(command: str)`: Executes a Linux command.
- `next_question()`: Retrieves the next question in the challenge.
- `read_output()`: Reads the output of the last executed command.

**Example Session:**

**User:** Take the next question and solve it by providing a linux command.

**Model:**
Thought: The first step is to get the next question from the challenge. I will use the `next_question` function to retrieve it.
Action: function calling next_question

**User:** (Model is called again with this input, after `next_question` runs)
Observation: the next question

**Model:**
Thought: The last step was to get the question. Now that I have it, I need to generate a command to solve it. My plan is to use a specific command on the target file.
Action: Function Calling ExecuteCmd(command='grep -i "some_pattern" Datasets/file_analysis_2/HDWallpaperEngine.txt')

**User:** (Model is called again with this input, after `ExecuteCmd` runs)
Observation: the output of the command

**Model:**
Thought: I have the output of the command. I need to analyze it to see if it contains the answer. If the answer is present, I will formulate the final Answer. If not, I will need to formulate a new plan and command to get the answer.
Action: function calling read_output()

**User:** (Model is called again with this input, after `read_output()` runs)
Observation: the output of the previous command

**Model:**
Answer: The answer to the question is '...' """.strip()

In [35]:
#create a chat session
chat = client.chats.create(
            history=[],
            model="gemini-2.5-flash",
            config=types.GenerateContentConfig(
                    # here are the external functions' declarations that the LLM will call to solve the problem
                    tools=[types.Tool(function_declarations=[ExecuteCmd_dec,next_question_dec,read_output_dec])], 
                    system_instruction=prompt3 #system prompt
                    
                ))
i=0
answer_list=["answer not found"]*len(questions)

while i<len(questions):
        
        # we limit the yser to enter "1", "2" or "3" to choose what to tell the model
        user_input = int(input("Enter 1 to solve the next question or 2 to further analyse the question: "))
        if user_input == 1:
            query = "Take the next question and solve it by providing a linux command always generate thought before calling"
            
        elif user_input == 2:
            query = "Analyze the previous command's output and generate a new command or return the final answer"

        elif user_input ==3:break    
        else:
            print("Invalid input. Please enter 1 or 2.")

        #loop 
        j=0 
        MaxAttempts=4
        next_input=query
        
        while j<MaxAttempts:
            j+=1
            #this is the request that triggers gemini
            response= chat.send_message(next_input)
            parts=response.candidates[0].content.parts[0]
            # Check for a function call
            if parts.function_call:

                tool_call = parts.function_call
                args_str = ", ".join(f"{k}={repr(v)}" for k, v in tool_call.args.items())
                print(f"Action: {tool_call.name}({args_str})")

                if tool_call.name == "ExecuteCmd":
                    result = ExecuteCmd(**tool_call.args)#there is break here
                    print(f"Observation: {result}")
                    break
                elif tool_call.name== "next_question":
                    result = next_question()
                else: result= read_output()

                next_input=f"Observation: {result}"
                print(next_input)
                
            else:

                print(parts.text)                
                match = re.search(r"(?i)answer\s*:\s*(.*)", parts.text)
                if match:
                    answer_list[i-1]=match.group(1).strip()  # Extract text after 'Answer:'
                    break

Action: next_question()
Observation: What is the SHA256 hash value for the PowerShell script file?
Action: ExecuteCmd(command_str='sha256sum Datasets/file_analysis_2/HDWallpaperEngine.txt')
Observation: Output saved in a file
Action: read_output()
Observation: e0b7a2ad2320ac32c262aeb6fe2c6c0d75449c6e34d0d18a531157c827b9754e  Datasets/file_analysis_2/HDWallpaperEngine.txt
The SHA256 hash value for the PowerShell script file is `e0b7a2ad2320ac32c262aeb6fe2c6c0d75449c6e34d0d18a531157c827b9754e`.
Action: next_question()
Observation: What email address is used to send and receive emails? (Format: Mailbox@domain.tld)
Action: ExecuteCmd(command_str="grep -oE '[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}' Datasets/file_analysis_2/HDWallpaperEngine.txt")
Observation: Output saved in a file
Action: read_output()
Observation: chaudhariparth454@gmail.com
chaudhariparth454@gmail.com
The email address used to send and receive emails is `chaudhariparth454@gmail.com`.
Action: next_question()
Observ

In [12]:
print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        function_call=FunctionCall(
          args={
            'command_str': 'sha256sum Datasets/file_analysis_2/HDWallpaperEngine.txt'
          },
          name='ExecuteCmd'
        ),
        thought_signature=b'\n\xb7\x01\x01T\xa8\\\xee\x13}\xee\xbbq\xa4\x15\xc5\xcf\x1c\x93\x94\xb6.f\nP\x82y\xb9U\x8a\xd8[\xcfBP\x93\xcb\xf9\xb4I\x84\xa2\xcc\xd2X\xc5\xb0\x06\x1b\xd8t\xbd\n\xa7eR\xca/\xfc\xea\x1b?\xe7\x99\x89\xae\x86\xc9\xd1\x02\xc6\xd5\x7f\xbf\x97\xb6z\x96\x14e\x16>\xce\x05cT\xa9s&\x8c\xcd\xc4\xd6\x16\xfbzo...'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None response_id=None model_version='gemini-2.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=34,
  prompt_token_count=818,
  prompt_tokens_details=[
    ModalityTokenCount(
      modal

In [15]:
print(answer_list)

['e0b7a2ad2320ac32c262aeb6fe2c6c0d75449c6e34d0d18a531157c827b9754e', 'chaudhariparth454@gmail.com', 'Parth@123', '587', 'user32.dll', 'temp']


In [22]:
for message in chat.get_history():
    print(message)

parts=[Part(
  text='Take the next question and solve it by providing a linux command always generate thought before calling'
)] role='user'
parts=[Part(
  text="I'll get the first question using `next_question` to begin the investigation.",
  thought_signature=b'\n\x93\x01\x01T\xa8\\\xee\xf2\xe1\xe1\xba\xf0\x13\x90\xed\x15H\xa7\xd1\xc8\x17\x95\x83*\x98T\xff\xfb\xf6\x19\x87{"U\xbc\xd1E\xa8\x13@\xf8\x0c6M~9\xa7\xc07C\xf2\xa6\x1c\xb3LO\xb4\x19\xf1O\xe9\x11[]>\xb1\x1c\xd9\xe2\xf3\xdb"|\'c\xbf8W,\xe8D{\xda\x08T\x00\x12\xdd<v\xd7\x97\xc1\xe80\xe0...'
), Part(
  function_call=FunctionCall(
    args={},
    name='next_question'
  )
)] role='model'
parts=[Part(
  text='Take the next question and solve it by providing a linux command always generate thought before calling'
)] role='user'
parts=[Part(
  text="I need to find the SHA256 hash of the file. I'll use the `sha256sum` command on the target file, `Datasets/file_analysis_2/HDWallpaperEngine.txt`, to compute the hash.",
  thought_signature

In [20]:

tools = types.Tool(function_declarations=[next_question_dec])
response=client.models.generate_content(
    model="gemini-2.5-flash",
    
    config = types.GenerateContentConfig(tools=[tools]),
    contents="what is the whether now"
)


In [25]:
for candidate in response.candidates:
    for part in candidate.content.parts:
        if hasattr(part, "text") and part.text:
            print(part.text)


I cannot tell you what the weather is right now, as I do not have access to real-time information.


In [27]:
print(response.candidates[0].content.parts[0].text.lower())

i cannot tell you what the weather is right now, as i do not have access to real-time information.


In [42]:
textt="answer: dida"
match = re.search(r"(?i)answer\s*:\s*(.*)", textt)
if match:
    answer_text = match.group(1).strip()  # Extract text after 'Answer:'
print(answer_text)

dida
